# NLP Homework 8
## Sentiment Analysis
### Julia Layne

In [5]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import re

## Perform a vocabulary-based sentiment analysis of the movie reviews

## Data Sources


## Technique

## Findings

In [34]:
reviews = pd.read_csv("parsed_reviews_sentiment.csv")
for i in range(6):
    print('---- Cluster ',i,'----')
    filtered = reviews['cluster']==i
    review_filtered = reviews.copy()
    review_filtered.where(filtered, inplace = True)
    review_filtered = review_filtered.dropna()
    #average, median, high, and low
    print('Average: ',review_filtered['SENTIMENT'].mean())
    print('Median: ',review_filtered['SENTIMENT'].median())
    print('High: ',review_filtered['SENTIMENT'].max())
    print('Low: ',review_filtered['SENTIMENT'].min())
    if review_filtered['SENTIMENT'].median() >0:
        print('Overall Positive')
    else:
        print('Overall Negative')
    print("Old Description:\n",oldDescription[i])
    

---- Cluster  0 ----
Average:  3.2083333333333335
Median:  2.3125
High:  11.25
Low:  -0.125
Overall Positive
Old Description:
 Mostly negative MidSized reviews. Even positives point out negatives
---- Cluster  1 ----
Average:  0.49375
Median:  0.5
High:  5.75
Low:  -3.5
Overall Positive
Old Description:
 Mostly negative and Luke-warm reviews
---- Cluster  2 ----
Average:  0.90625
Median:  0.1875
High:  4.0
Low:  -0.75
Overall Positive
Old Description:
 Short reviews, positive for the dark comedy and negative on the regular comedies
---- Cluster  3 ----
Average:  1.2022068965517243
Median:  1.25
High:  8.25
Low:  -6.375
Overall Positive
Old Description:
 Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'
---- Cluster  4 ----
Average:  0.25
Median:  -0.0625
High:  5.0
Low:  -3.5
Overall Negative
Old Description:
 Mostly Positive, Longer reviews
---- Cluster  5 ----
Average:  -1.375
Median:  -1.375
High:  -0.375
Low: 

# Code

### 1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.

In [28]:
reviews = pd.read_csv("parsed_reviews_clustered6.csv")
reviews["SENTIMENT"] = '-'

In [29]:
#Sourced from this article with some changes to get the swap from pennbank 
#https://nlpforhackers.io/sentiment-analysis-intro/

lemmatizer = WordNetLemmatizer()

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    if tag.startswith('N'):
        return wn.NOUN
    if tag.startswith('R'):
        return wn.ADV
    if tag.startswith('V'):
        return wn.VERB
    return None
 
def clean_text(text):
    text = re.sub(r'[^\w\s]',' ',text)
    return text
 
 
def swn_polarity(text):
    sentiment = 0.0
    tokens_count = 0
    text = clean_text(text)
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    #return the overall sentiment instead of the 0/1 for positive negative to allow for handling later
    return sentiment

In [30]:
# Since we're shuffling, you'll get diffrent results
print(swn_polarity(reviews["CONTENT"][0]))

-0.75


In [31]:
for i in reviews.index:
    text = reviews["CONTENT"][i]
    sentiment = swn_polarity(text)
    #print(sentiment)
    reviews["SENTIMENT"][i] = sentiment

<ipython-input-31-b93bb20e8b2d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["SENTIMENT"][i] = sentiment


In [33]:
reviews.to_csv(r'parsed_reviews_sentiment.csv', index = False)

In [32]:
reviews.head()

,URL,SOUP,CONTENT,NP_CHUNK,cluster,SENTIMENT
0,https://www.imdb.com/review/rw6540019/?ref_=tt...,"\n<!DOCTYPE html>\n\n<html xmlns:fb=""http://ww...",It might have been an error on Disney's part t...,"['It', 'an', 'error', 'Disney', 'part', 'episo...",2,-0.75
1,https://www.imdb.com/review/rw6562703/?ref_=tt...,"\n<!DOCTYPE html>\n\n<html xmlns:fb=""http://ww...",Writing this with bated breath waiting on epis...,"['bated', 'breath', 'episode', 'The', 'whole',...",0,2
2,https://www.imdb.com/review/rw6650913/?ref_=tt...,"\n<!DOCTYPE html>\n\n<html xmlns:fb=""http://ww...",One might wonder that the above adjectives do ...,"['the', 'above', 'adjectives', 'the', 'Marvel'...",1,-3.5
3,https://www.imdb.com/review/rw6603814/?ref_=tt...,"\n<!DOCTYPE html>\n\n<html xmlns:fb=""http://ww...",I can't speak for the reviews that bash this s...,"['I', 'the', 'reviews', 'this', 'show', 'three...",0,2.625
4,https://www.imdb.com/review/rw6479388/?ref_=tt...,"\n<!DOCTYPE html>\n\n<html xmlns:fb=""http://ww...",What a waste $25 Million Dollars A episode don...,"['a', 'waste', '25', 'Million', 'Dollars', 'A'...",0,-0.125


### 2.	 For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.

In [24]:
oldDescription = [
    "Mostly negative MidSized reviews. Even positives point out negatives",
    "Mostly negative and Luke-warm reviews",
    "Short reviews, positive for the dark comedy and negative on the regular comedies",
    "Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'",
    "Mostly Positive, Longer reviews",
    "Two short reviews for I Blame Society - Comedy Horror"
]

In [25]:
for i in range(6):
    print('---- Cluster ',i,'----')
    filtered = reviews['cluster']==i
    review_filtered = reviews.copy()
    review_filtered.where(filtered, inplace = True)
    review_filtered = review_filtered.dropna()
    #average, median, high, and low
    print('Average: ',review_filtered['SENTIMENT'].mean())
    print('Median: ',review_filtered['SENTIMENT'].median())
    print('High: ',review_filtered['SENTIMENT'].max())
    print('Low: ',review_filtered['SENTIMENT'].min())
    if review_filtered['SENTIMENT'].median() >0:
        print('Overall Positive')
    else:
        print('Overall Negative')
    print("Old Description:\n",oldDescription[i])
    


---- Cluster  0 ----
Average:  3.2083333333333335
Median:  2.3125
High:  11.25
Low:  -0.125
Overall Positive
Old Description:
 Mostly negative MidSized reviews. Even positives point out negatives
---- Cluster  1 ----
Average:  0.49375
Median:  0.5
High:  5.75
Low:  -3.5
Overall Positive
Old Description:
 Mostly negative and Luke-warm reviews
---- Cluster  2 ----
Average:  0.90625
Median:  0.1875
High:  4.0
Low:  -0.75
Overall Positive
Old Description:
 Short reviews, positive for the dark comedy and negative on the regular comedies
---- Cluster  3 ----
Average:  1.2022068965517243
Median:  1.25
High:  8.25
Low:  -6.375
Overall Positive
Old Description:
 Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'
---- Cluster  4 ----
Average:  0.25
Median:  -0.0625
High:  5.0
Low:  -3.5
Overall Negative
Old Description:
 Mostly Positive, Longer reviews
---- Cluster  5 ----
Average:  -1.375
Median:  -1.375
High:  -0.375
Low: 

### 3.	For extra credit, analyze sentiment of chunks as follows:
 - a.	Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).
 - b.	Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see. 


### Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings.